## PySpark 설치

In [ ]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 처리할 데이터 파일을 먼저 다운로드 받아온다

In [1]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/1800.csv

--2023-07-04 12:11:41--  https://s3-geospatial.s3-us-west-2.amazonaws.com/1800.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 3.5.83.169, 3.5.84.174, 3.5.81.19, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|3.5.83.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62728 (61K) [text/csv]
Saving to: ‘1800.csv’

1800.csv            100%[===================>]  61.26K   388KB/s    in 0.2s    

2023-07-04 12:11:41 (388 KB/s) - ‘1800.csv’ saved [62728/62728]



In [2]:
!ls -tl

total 232
-rw-r--r-- 1 jovyan users 27540 Jul  4 12:06 PySpark_설치_및_테스트.ipynb
-rw-r--r-- 1 jovyan users 22056 Jul  4 12:06 PySpark_DataFrame_5.ipynb
-rw-r--r-- 1 jovyan users 32483 Jul  4 12:06 PySpark_DataFrame_4.ipynb
-rw-r--r-- 1 jovyan users 14392 Jul  4 12:06 PySpark_DataFrame_3.ipynb
-rw-r--r-- 1 jovyan users 25291 Jul  4 12:06 PySpark_DataFrame_2.ipynb
-rw-r--r-- 1 jovyan users 37488 Jul  4 12:06 PySpark_DataFrame_1.ipynb
-rw-r--r-- 1 jovyan users 62728 Apr 10  2022 1800.csv


In [3]:
!head -5 1800.csv

ITE00100554,18000101,TMAX,-75,,,E,
ITE00100554,18000101,TMIN,-148,,,E,
GM000010962,18000101,PRCP,0,,,E,
EZE00100082,18000101,TMAX,-86,,,E,
EZE00100082,18000101,TMIN,-135,,,E,


## 판다스 데이터프레임으로 처리해본다

In [ ]:
import pandas as pd 

pd_df = pd.read_csv(
    "1800.csv",
    names=["stationID", "date", "measure_type", "temperature"],
    usecols=[0, 1, 2, 3]
)

In [ ]:
pd_df.head()

,stationID,date,measure_type,temperature
0,ITE00100554,18000101,TMAX,-75
1,ITE00100554,18000101,TMIN,-148
2,GM000010962,18000101,PRCP,0
3,EZE00100082,18000101,TMAX,-86
4,EZE00100082,18000101,TMIN,-135


In [ ]:
# Filter out all but TMIN entries
pd_minTemps = pd_df[pd_df['measure_type'] == "TMIN"]

In [ ]:
pd_minTemps.head()

In [ ]:
# Select only stationID and temperature
pd_stationTemps = pd_minTemps[["stationID", "temperature"]]

In [ ]:
# Aggregate to find minimum temperature for every station
pd_minTempsByStation = pd_stationTemps.groupby(["stationID"]).min("temperature")
pd_minTempsByStation.head()

,temperature
stationID,
EZE00100082,-135
ITE00100554,-148


## Spark으로 처리해본다

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name", "PySpark DataFrame #1")
conf.set("spark.master", "local[*]")

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

In [ ]:
df = spark.read.format("csv").load("1800.csv") # spark.read.csv("1800.csv")

In [ ]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [ ]:
df = spark.read.format("csv")\
    .load("1800.csv")\
    .toDF("stationID", "date", "measure_type", "temperature", "_c4", "_c5", "_c6", "_c7")

In [ ]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: string (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [ ]:
df = spark.read.format("csv")\
    .option("inferSchema", "true")\
    .load("1800.csv")\
    .toDF("stationID", "date", "measure_type", "temperature", "_c4", "_c5", "_c6", "_c7")

In [ ]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [ ]:
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.sql.types import StructType, StructField

schema = StructType([ \
                     StructField("stationID", StringType(), True), \
                     StructField("date", IntegerType(), True), \
                     StructField("measure_type", StringType(), True), \
                     StructField("temperature", FloatType(), True)])

In [ ]:
# df = spark.read.schema(schema).format("csv").load("1800.csv")
df = spark.read.schema(schema).csv("1800.csv")

In [ ]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)



In [ ]:
# Filter out all but TMIN entries
minTemps = df.filter(df.measure_type == "TMIN")

In [ ]:
minTemps.count()

730

In [ ]:
# Column expression으로 필터링 적용
minTemps = df.where(df.measure_type == "TMIN")

In [ ]:
minTemps.count()

730

In [ ]:
# SQL expression으로 필터링 적용
minTemps = df.where("measure_type = 'TMIN'")

In [ ]:
minTemps.count()

730

In [ ]:
# Aggregate to find minimum temperature for every station
minTempsByStation = minTemps.groupBy("stationID").min("temperature")
minTempsByStation.show()

+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+



In [ ]:
# Select only stationID and temperature
stationTemps = minTemps[["stationID", "temperature"]]

In [ ]:
stationTemps.show(5)

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
+-----------+-----------+
only showing top 5 rows



In [ ]:
stationTemps = minTemps.select("stationID", "temperature")

In [ ]:
stationTemps.show(5)

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
+-----------+-----------+
only showing top 5 rows



In [ ]:
stationTemps.show(5)

+-----------+-----------+-----------+-----------+
|  stationID|  stationID|  stationID|  stationID|
+-----------+-----------+-----------+-----------+
|ITE00100554|ITE00100554|ITE00100554|ITE00100554|
|EZE00100082|EZE00100082|EZE00100082|EZE00100082|
|ITE00100554|ITE00100554|ITE00100554|ITE00100554|
|EZE00100082|EZE00100082|EZE00100082|EZE00100082|
|ITE00100554|ITE00100554|ITE00100554|ITE00100554|
+-----------+-----------+-----------+-----------+
only showing top 5 rows



In [ ]:
# Collect, format, and print the results
results = minTempsByStation.collect()

In [ ]:
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	-148.00F
EZE00100082	-135.00F


## Spark SQL로 처리해보기

In [ ]:
df.createOrReplaceTempView("station1800")

In [ ]:
results = spark.sql("""SELECT stationID, MIN(temperature)
FROM station1800
WHERE measure_type = 'TMIN'
GROUP BY 1""").collect()

In [ ]:
# pyspark.sql.Row는 DataFrame의 레코드에 해당하며 필드별로 이름이 존재# 
for r in results:
    print(r)

Row(stationID='ITE00100554', min(temperature)=-148.0)
Row(stationID='EZE00100082', min(temperature)=-135.0)
